In [58]:
import torch
import torchaudio
from torchaudio.pipelines import MMS_FA as bundle
from typing import List
import IPython
import matplotlib.pyplot as plt
from pypinyin import pinyin, lazy_pinyin, Style

print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import whisper
from pydub import AudioSegment
import librosa
from opencc import OpenCC


2.4.0+cu118
2.4.0+cu118
cuda


In [59]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

raw_data = "raw_audio\\Spring.WAV"

model = whisper.load_model("large-v2")
# 使用Whisper進行語音識別
result = model.transcribe(raw_data)
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 4.2,
  'text': '很多人在買股票時都不知道何時該買入賣出',
  'tokens': [50364,
   20778,
   4035,
   3581,
   31001,
   14356,
   94,
   41925,
   6611,
   7182,
   17572,
   11906,
   6611,
   22033,
   31001,
   14028,
   13352,
   96,
   7781,
   50574],
  'temperature': 0.0,
  'avg_logprob': -0.12928757213410877,
  'compression_ratio': 1.4356435643564356,
  'no_speech_prob': 0.006587646901607513},
 {'id': 1,
  'seek': 0,
  'start': 4.2,
  'end': 5.8,
  'text': '會感到很迷惘',
  'tokens': [50574, 6236, 9709, 4511, 4563, 3316, 115, 4799, 246, 50654],
  'temperature': 0.0,
  'avg_logprob': -0.12928757213410877,
  'compression_ratio': 1.4356435643564356,
  'no_speech_prob': 0.006587646901607513},
 {'id': 2,
  'seek': 0,
  'start': 5.8,
  'end': 9.5,
  'text': '而學習支撐壓力能幫助我們把握到良好的買入點',
  'tokens': [50654,
   11070,
   21372,
   34025,
   24400,
   20559,
   238,
   45964,
   13486,
   8225,
   32187,
   37618,
   5884,
   16075,
   11673,
   94,
   4511,
   33950,
 

In [60]:
sentence =[]
cc = OpenCC('s2t')

for i in range(len(result['segments'])):
    sentence.append([cc.convert(result['segments'][i]['text'].lower().replace('》','').replace('《','').replace('%','').replace('。','').replace('?','').replace('【','').replace('】','').replace('-','').replace('.','').replace(',', '').replace('6','六').replace('4','四').replace('2','二').replace('9','九').replace('8','八').replace('5','五').replace('3','三').replace('0','零').replace('1','一').replace('7','七').replace(' ','').replace('、','')), 
                     result['segments'][i]['start'], 
                     result['segments'][i]['end']])
   

sentence

[['很多人在買股票時都不知道何時該買入賣出', 0.0, 4.2],
 ['會感到很迷惘', 4.2, 5.8],
 ['而學習支撐壓力能幫助我們把握到良好的買入點', 5.8, 9.5],
 ['買賣操作都有依據', 9.5, 11.4],
 ['這支影片我會用最簡單易懂的方式', 11.4, 13.9],
 ['由淺入深地教大家支撐和壓力', 13.9, 16.6],
 ['首先會教大家支撐和壓力的定義和概念', 16.6, 19.900000000000002],
 ['然後會教如何在股票的走勢圖上', 19.900000000000002, 22.6],
 ['畫出重要的支撐和壓力區', 22.6, 25.2],
 ['最後會講解在什麼情況下', 25.2, 27.1],
 ['支撐和壓力容易被突破', 27.1, 29.1],
 ['這部分會延伸講解到價格形態', 29.3, 31.700000000000003],
 ['大家看完後就會更明白', 31.700000000000003, 33.4],
 ['為什麼美國投資冠軍mark', 33.4, 35.300000000000004],
 ['常用的vcp形態', 35.300000000000004, 36.800000000000004],
 ['是一種非常適合做多的價格形態', 36.800000000000004, 39.5],
 ['如果你想學好怎樣分析支撐壓力', 39.5, 41.900000000000006],
 ['和把握到良好的買入點', 41.900000000000006, 43.900000000000006],
 ['就記得把影片看到最後', 43.900000000000006, 45.8],
 ['因為最後這部分是最精華和最重要的內容', 45.8, 49.2],
 ['壓力可以定義為這一段期間內', 50.5, 52.900000000000006],
 ['阻止價格繼續上漲的賣壓', 52.900000000000006, 55.3],
 ['也就是說當股價漲到某個位置後', 55.3, 57.8],
 ['如果賣壓變得非常大', 57.8, 59.599999999999994],
 ['像有個天花板一樣', 59.599999999999994, 61.09

In [61]:
# del  sentence[-1]
# sentence

In [62]:
audio = AudioSegment.from_file(raw_data)
for i in range(len(sentence)):
    audio_clip = audio[sentence[i][1] *1000: sentence[i][2]*1000]
    audio_clip.export(f"sentences\\{(sentence[i][0].lower())}.wav", format="wav")

In [63]:
model = bundle.get_model()
model.to(device)

tokenizer = bundle.get_tokenizer()
aligner = bundle.get_aligner()

def compute_alignments(waveform: torch.Tensor, transcript: List[str]):
    with torch.inference_mode():
        emission, _ = model(waveform.to(device))
        token_spans = aligner(emission[0], tokenizer(transcript))
    return emission, token_spans

def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)


def plot_alignments(waveform, token_spans, emission, transcript, sample_rate=bundle.sample_rate):
    ratio = waveform.size(1) / emission.size(1) / sample_rate

    fig, axes = plt.subplots(2, 1)
    axes[0].imshow(emission[0].detach().cpu().T, aspect="auto")
    axes[0].set_title("Emission")
    axes[0].set_xticks([])

    axes[1].specgram(waveform[0], Fs=sample_rate)
    for t_spans, chars in zip(token_spans, transcript):
        t0, t1 = t_spans[0].start, t_spans[-1].end
        axes[0].axvspan(t0 - 0.5, t1 - 0.5, facecolor="None", hatch="/", edgecolor="white")
        axes[1].axvspan(ratio * t0, ratio * t1, facecolor="None", hatch="/", edgecolor="white")
        axes[1].annotate(f"{_score(t_spans):.2f}", (ratio * t0, sample_rate * 0.51), annotation_clip=False)

        for span, char in zip(t_spans, chars):
            t0 = span.start * ratio
            axes[1].annotate(char, (t0, sample_rate * 0.55), annotation_clip=False)

    axes[1].set_xlabel("time [second]")
    fig.tight_layout()

def preview_word(waveform, spans, num_frames, transcript, sample_rate):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    #print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    time_StarAndEnd = [ x0 / sample_rate, x1/sample_rate] # 回傳單個字的起始時間與結束時間
    segment = waveform[:, x0:x1]
    #return IPython.display.Audio(segment.numpy(), rate=sample_rate)
    return time_StarAndEnd
    

In [64]:
for i in range(0,len(sentence)):#len(sentence)-1
    text_normalized = ' '.join(lazy_pinyin(sentence[i][0]))#將文字轉為沒有音調的拼音，lazy_pinyin是陣列所以要再join成字串

    waveform, sample_rate = librosa.load(f"sentences\\{sentence[i][0]}.wav")
    waveform_tensor = torch.tensor(waveform).unsqueeze(0)

    transcript = text_normalized.split()
    emission, token_spans = compute_alignments(waveform_tensor, transcript)
    num_frames = emission.size(1)


    #plot_alignments(waveform, token_spans, emission, transcript)

    print("Raw Transcript: ", sentence[i][0])
    print("Normalized Transcript: ", text_normalized)
    IPython.display.Audio(waveform, rate=sample_rate)

    text_raw = sentence[i][0]
    word_start_end = []
    pinyin_tone = pinyin(text_raw, style=Style.TONE3, heteronym=False)
    for j in range(len(transcript)):#len(transcript)
        timeStartEnd = preview_word(waveform_tensor, token_spans[j], num_frames, transcript[j], sample_rate)
        word_start_end.append([pinyin_tone[j][0], timeStartEnd[0], timeStartEnd[1]])
    print(word_start_end)

    audio = AudioSegment.from_file(f"sentences\\{sentence[i][0]}.wav")
    file_name = sentence[i][0]
    for k in range(len(word_start_end)):
        segment_audio = audio[word_start_end[k][1] *1000: word_start_end[k][2]*1000]
        segment_audio.export(f"data\\{file_name}-{k}_{word_start_end[k][0]}.wav", format="wav")
    print('------------------------------------------')

Raw Transcript:  很多人在買股票時都不知道何時該買入賣出
Normalized Transcript:  hen duo ren zai mai gu piao shi dou bu zhi dao he shi gai mai ru mai chu
[['hen3', 0.6975510204081633, 0.7992743764172335], ['duo1', 0.8283446712018141, 0.9155555555555556], ['ren2', 0.944625850340136, 1.0463492063492064], ['zai4', 1.0899319727891157, 1.2207256235827664], ['mai3', 1.2643537414965986, 1.3951473922902495], ['gu3', 1.4678004535147393, 1.5259410430839002], ['piao4', 1.59859410430839, 1.77297052154195], ['shi2', 1.8311111111111111, 1.9328344671201814], ['dou1', 2.063628117913832, 2.1799092970521543], ['bu4', 2.2234920634920634, 2.2816326530612243], ['zhi1', 2.325215419501134, 2.3979138321995466], ['dao4', 2.4414965986394557, 2.5722902494331064], ['he2', 2.674013605442177, 2.7612244897959184], ['shi2', 2.906530612244898, 3.0082993197278913], ['gai1', 3.0954648526077095, 3.2262585034013607], ['mai3', 3.3134693877551022, 3.4588208616780047], ['ru4', 3.5169160997732427, 3.6041269841269843], ['mai4', 3.720408163265306,

: 